In [2]:
import numpy as np
from numpy import linalg as LA
from ncon import ncon
import matplotlib.pyplot as plt
from operator import itemgetter

In [4]:
h = 5
J = 3
sigma_x = np.array([[0,1],[1,0]])
sigma_z = np.array([[1,0],[0,-1]])
I = np.eye(2)
H = h*np.tensordot(sigma_x, np.tensordot(I,I, axes=0), axes=0) + J*np.tensordot(sigma_z, np.tensordot(sigma_z,I, axes=0), axes=0) + h*np.tensordot(I, np.tensordot(sigma_x,I, axes=0), axes=0) + J*np.tensordot(I, np.tensordot(sigma_z,sigma_z, axes=0), axes=0)
H = H.reshape((8,8))
H

array([[ 6.,  0.,  0.,  0.,  5.,  0.,  0.,  5.],
       [ 5.,  0.,  0.,  5., -6.,  0.,  0.,  0.],
       [ 5.,  0.,  0.,  5.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  5.,  0.,  0.,  5.],
       [ 5.,  0.,  0.,  5.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  5.,  0.,  0.,  5.],
       [ 0.,  0.,  0., -6.,  5.,  0.,  0.,  5.],
       [ 5.,  0.,  0.,  5.,  0.,  0.,  0.,  6.]])

In [5]:
w,v = np.linalg.eig(H)
w = abs(w)
lambda_min = min(enumerate(w), key=itemgetter(1))[1]
index = min(enumerate(w), key=itemgetter(1))[0]
min_state = v[index]
min_state

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.17165167,
        0.53545511, -0.17165167, -0.53545511])

In [6]:
Null = np.array([[0,0],[0,0]])
H_left = np.array([h*sigma_x, J*sigma_z, I]).transpose((1,0,2))
H_mid = np.array([[I, Null, Null], 
         [sigma_z, Null, Null],
         [h*sigma_x, J*sigma_z, I]]).transpose((0,2,1,3))
H_right = np.array([I, sigma_z, h*sigma_x])


In [63]:
n = 2
r = 3
# G1 = np.random.rand(r,n)
# G2 = np.random.rand(r,r,n)
# G3 = np.random.rand(r,n)
G1 = np.array([[1,2],[3,4],[5,6]])
G2 = np.array([[[1,0],[0,1],[0,0]],[[1,1],[3,1],[2,1]],[[4,1],[6,4],[0,1]]])
G3 = np.array([[3,1],[2,6],[5,4]])
E = np.eye(n)
E3 = np.array([E, E, E])
E4 = np.array([[E, Null, Null],[Null, E, Null],[Null, Null, E]]).transpose((0,2,1,3))

In [70]:
# dL/dG1:
A = np.einsum('abd,be,cfh,fdgi,gej,kil,lj->cahk',G2.conj(),G3.conj(),H_left,H_mid,H_right,G2.transpose((0,2,1)),G3)
B = np.einsum('abd,be,cfh,fdgi,gej,kil,lj->cahk',G2.conj(),G3.conj(),E3.transpose((1,0,2)),E4,E3,G2.transpose((0,2,1)),G3)
A = A.reshape((n*r,n*r))
B = B.reshape((n*r,n*r))
C = np.linalg.inv(B) * A

In [74]:
w,v = np.linalg.eig(C)
w = abs(w)
lambda_min = min(enumerate(w), key=itemgetter(1))[1]
index = min(enumerate(w), key=itemgetter(1))[0]
min_state = v[index]
G1 = min_state.reshape((r,n))       #update G1

In [75]:
G1.shape

(3, 2)

In [88]:
# dL/dG2:
A = np.einsum('ac,be,cfh,fdgi,gej,hk,lj->adbkil',G1.conj(),G3.conj(),H_left,H_mid,H_right,G1.transpose((1,0)),G3)
B = np.einsum('ac,be,cfh,fdgi,gej,hk,lj->adbkil',G1.conj(),G3.conj(),E3.transpose((1,0,2)),E4,E3,G1.transpose((1,0)),G3)
A = A.reshape((n*r**2,n*r**2))
B = B.reshape((n*r**2,n*r**2))
C = np.linalg.inv(B) * A

LinAlgError: Singular matrix

In [80]:
B

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.   

In [ ]:
w,v = np.linalg.eig(C)
w = abs(w)
lambda_min = min(enumerate(w), key=itemgetter(1))[1]
index = min(enumerate(w), key=itemgetter(1))[0]
min_state = v[index]
G2 = min_state.reshape((r,n))       #update G2

In [81]:
G2.shape

(3, 3, 2)

In [84]:
# dL/dG3:
A = np.einsum('ac,abd,cfh,fdgi,gej,hk,kil->belj',G1.conj(),G2.conj(),H_left,H_mid,H_right,G1.transpose((1,0)),G2.transpose((0,2,1)))
B = np.einsum('ac,abd,cfh,fdgi,gej,hk,kil->belj',G1.conj(),G2.conj(),E3.transpose((1,0,2)),E4,E3,G1.transpose((1,0)),G2.transpose((0,2,1)))
A = A.reshape((n*r,n*r))
B = B.reshape((n*r,n*r))
C = np.linalg.inv(B) * A

In [85]:
w,v = np.linalg.eig(C)
w = abs(w)
lambda_min = min(enumerate(w), key=itemgetter(1))[1]
index = min(enumerate(w), key=itemgetter(1))[0]
min_state = v[index]
G3 = min_state.reshape((r,n))       #update G3

In [87]:
G3.shape

(3, 2)